In [1]:
using JLD, Plots, Combinatorics
using Base.Iterators
gr()

Plots.GRBackend()

In [2]:
arxname="../JLDFiles/Evocada/CSDEvocada_3R7.jld"
data=load(arxname)

Dict{String,Any} with 2 entries:
  "CSDPlanchada" => Dict{AbstractString,Array}("Trancazo_2"=>[-18.0116 -6.93308…
  "CSD"          => Dict{AbstractString,Array}("Trancazo_2"=>[-26.8791 -3.68286…

In [3]:
freq=7.022
csd=(data["CSD"]["Trancazo_1"]+data["CSD"]["Trancazo_2"]+data["CSD"]["Trancazo_3"])./3
(h,w,nmax)=size(csd)
epsi=21
  ss=[-1,0,1]

3-element Array{Int64,1}:
 -1
  0
  1

In [4]:
function simboliza(x, delta=epsi)
    result=0
    if x>delta
        result=1
    elseif x< -delta
        result=-1
    else
        result=0
    end
    return result
end

simboliza (generic function with 2 methods)

In [5]:
csdsymb=map(x->simboliza(x), csd);

In [6]:
function problock(xxs::Array, yys::Array)
    # calculamos la prob exp de encontrar el array yys en el array xxs
    lx=length(xxs)
    ly=length(yys)
    lp=lx-ly+1
    result=0
    if(lx<ly)
        error("tuabueliuta")
    else
        for j=1:lp
            if xxs[j:j+ly-1]==yys
                result+=1
            end
        end
    end
    return result/lp
end

problock (generic function with 1 method)

In [7]:
function ncartprod(ss, n)
    argumento=fill(ss, n)
    result=collect(product(argumento...))
    return(result)
end

ncartprod (generic function with 1 method)

In [41]:
function ncartprodR(ss, n)
# okey, esto no funciona nada bien. Muuuy lento y usa mas RAM
    result=[]
    if n==1
        for s in ss
            push!(result, [s])
        end
        return result
    else
        aux=ncartprodR(ss, n-1)
        for w in aux
            for s in ss
                vv=vcat(s, w)
                push!(result, vv)
            end
        end
        return result
    end
end

ncartprodR (generic function with 1 method)

In [38]:
@time a=ncartprodR(ss, 15);

114.976364 seconds (451.99 M allocations: 20.946 GiB, 47.14% gc time)


In [40]:
@time ncartprod(ss, 15);

  2.755678 seconds (598.34 k allocations: 1.628 GiB, 65.02% gc time)


In [66]:
# con permiso de StackOverflow oooo.key takes lot to compile!
issubvec(v,big) = any([v == big[i:(i+length(v)-1)] for i=1:(length(big)-length(v)+1)])



issubvec (generic function with 1 method)

In [127]:
function allprobsn(xxs, ss, n)
    result=Dict{Array, Float64}()
    palabras=ncartprod(ss, n)
    malas=Set()
    for p in palabras
        pr=[p...]
        aux=problock(xxs, pr)
        if aux>0
            result[pr]= aux
        else
            push!(malas, pr)
        end
    end
    return (result, malas)
end


function allprobsn(xxs, ss, n, vacias)
    
    #ush tambien muy ineficiente. Sigue siendo mejor buscar a lo pendejo???
            result=Dict{Array, Float64}()
    palabras=ncartprod(ss, n)
    malas=Set()
    for p in palabras
        pr=[p...]
        tanteada=any(issubvec(v, pr) for v in vacias)
        if tanteada
         #   print(pr, " esta tanteada!!  ")
            aux=0
        else
            aux=problock(xxs, pr)
            if aux>0
                result[pr]= aux
            else
                push!(malas, pr)
            end
        end # sobre tanteada
    end #sobre palabras
    return (result, malas)
end

allprobsn (generic function with 2 methods)

In [50]:
r=allprobsn(xs, ss, 5);
a=0
for q in keys(r)
    if r[q]==0
      #  println("yea zero ", q)
        a+=1
    end
end


221


In [88]:
function allprobsR(xxs, ss, n)
#not reaaalyy good. Actually preeeety bad
     result=Dict{Array, Float64}()
    if n==1
        palabras=ncartprod(ss, n)
        for p in palabras
            pr=[p...]
            result[pr]=problock(xxs, pr)
        end
    
    else
        
        auxres=allprobsR(xxs,ss,n-1)
        vacias=Set()
        for v in keys(auxres)
            if auxres[v]==0
                push!(vacias, v)
            end
        end
        
        palabras=ncartprod(ss, n)
        for p in palabras
            pr=[p...]
            tienemala=any(issubvec(v, pr) for v in vacias)
            if tienemala
                result[pr]=0
            else
                result[pr]=problock(xxs, pr)
            end
        end
        
        end  # cierra sobre n
    
    return result
end
                
                
    
    

allprobsR (generic function with 1 method)

In [87]:
@time q=allprobsR(xs,ss,8)

  2.798924 seconds (27.75 M allocations: 2.282 GiB, 21.37% gc time)


Dict{Array,Float64} with 6561 entries:
  [-1, 0, 1, -1, -1, 0, 1, 0]  => 0.0
  [-1, 1, 1, 0, 1, -1, 0, 0]   => 0.0
  [0, -1, 0, 0, 0, -1, 1, 1]   => 0.0
  [1, -1, 0, 1, 0, 1, -1, 1]   => 0.0
  [1, 1, 1, -1, -1, 0, -1, 1]  => 0.0
  [-1, 1, 0, 0, 0, 1, -1, -1]  => 0.0
  [1, 1, 0, -1, -1, 1, -1, 1]  => 0.0
  [0, 1, 0, -1, 1, 0, 1, 0]    => 0.0
  [0, 1, 1, 1, 1, 1, 0, 1]     => 0.0
  [0, 1, 1, 0, -1, -1, 1, 1]   => 0.0
  [-1, 1, 0, 1, 1, 1, 0, -1]   => 0.0
  [-1, 1, 0, 0, 1, 1, -1, -1]  => 0.0
  [-1, 0, 0, 0, -1, 1, 1, -1]  => 0.0
  [-1, -1, 1, 1, 0, 1, 1, -1]  => 0.0
  [0, -1, -1, -1, 0, 0, 0, -1] => 0.0
  [-1, 1, 0, 0, 1, 1, -1, 0]   => 0.0
  [-1, -1, 0, -1, 1, 1, 0, 0]  => 0.0
  [0, 0, -1, 1, 0, -1, 1, 0]   => 0.0
  [0, 0, -1, 1, 0, -1, 1, 1]   => 0.0
  [1, -1, 1, -1, 0, 1, 1, 0]   => 0.0
  [0, -1, 0, -1, 1, 0, 1, 1]   => 0.0
  [1, 1, 0, 0, -1, -1, -1, 0]  => 0.0
  [0, 0, 1, 1, 1, -1, -1, 1]   => 0.0
  [1, 1, -1, 0, -1, 1, 1, 1]   => 0.0
  [-1, 1, 1, 0, 1, -1, 0, -1]  => 0.0
  ⋮        

In [163]:
@time (p, v5)=allprobsn(xs, ss, 5)

  0.002697 seconds (60.56 k allocations: 7.326 MiB)


(Dict{Array,Float64}([0, 0, 0, 0, 1]=>0.0122951,[-1, 0, -1, -1, 0]=>0.00409836,[0, 0, 1, 0, 0]=>0.00819672,[0, -1, -1, 0, 0]=>0.00819672,[-1, 0, 0, 0, 0]=>0.0122951,[0, 0, 0, 1, 1]=>0.00409836,[0, 0, 0, 0, -1]=>0.0122951,[0, 0, -1, 0, 0]=>0.00409836,[0, -1, 0, 0, 0]=>0.00409836,[0, 0, 0, -1, 0]=>0.00409836…), Set(Any[[0, -1, 1, 0, 1], [-1, 1, 0, -1, -1], [-1, -1, 0, 0, -1], [1, 1, 1, 0, 1], [1, 0, 1, 1, 0], [-1, -1, 0, 1, -1], [-1, -1, -1, 0, 1], [-1, 0, 1, -1, 0], [0, 0, 1, 1, 1], [1, -1, 1, 0, -1]  …  [0, 1, 1, -1, -1], [-1, 0, 1, 0, 1], [0, -1, 0, 1, -1], [0, 0, -1, 1, 1], [0, 1, 1, 0, -1], [-1, 0, 0, 1, -1], [-1, -1, 1, 0, -1], [-1, -1, 1, -1, 1], [-1, 1, 0, -1, 1], [1, -1, -1, -1, 0], [0, 1, 0, -1, 0]]))

In [160]:
@time (p, v)=allprobsn(xs, ss, 9)

  0.489057 seconds (4.85 M allocations: 730.258 MiB, 43.50% gc time)


(Dict{Array,Float64}([0, 0, 0, 0, -1, -1, 0, -1, -1]=>0.00416667,[0, 0, 0, 0, -1, -1, 0, 0, 0]=>0.00416667,[0, 0, 0, 0, 0, 0, 1, 0, 0]=>0.00833333,[0, 0, 1, 1, 0, 0, 0, 0, 0]=>0.00416667,[0, 0, 0, -1, -1, 0, -1, -1, 0]=>0.00416667,[0, 0, 0, -1, -1, 0, 0, 0, 0]=>0.00416667,[1, 0, 0, 0, 0, 0, -1, 0, 0]=>0.00416667,[0, 0, -1, 0, 0, 0, 0, 0, 0]=>0.00416667,[0, 0, 0, 1, 0, 0, 0, 0, 0]=>0.00833333,[0, 0, 0, 0, 0, 0, 0, -1, -1]=>0.00833333…), Set(Any[[0, -1, -1, 1, 0, -1, 1, 0, -1], [-1, 0, 0, 0, -1, -1, -1, -1, 0], [1, 0, 1, 0, -1, -1, 0, -1, 1], [-1, -1, 0, 1, 1, 1, 1, 1, -1], [-1, 1, 0, 1, -1, -1, 1, 1, 0], [1, 0, -1, -1, -1, 1, 1, 0, 0], [-1, 1, 0, 1, -1, 0, 0, -1, -1], [1, 1, -1, 1, 0, 0, -1, 0, 0], [0, -1, 0, -1, 1, -1, 0, 0, 0], [0, 0, 1, -1, -1, 1, 0, -1, -1]  …  [0, 0, 0, 0, 0, 1, 0, 1, 1], [-1, 0, -1, 1, 1, 1, -1, 1, 1], [0, 0, 0, 0, 1, 1, 0, 0, 1], [1, 0, 0, 1, -1, -1, 1, -1, 0], [1, 0, 0, -1, 0, 0, 0, 0, 1], [1, 0, -1, -1, -1, -1, 0, 1, -1], [-1, -1, -1, 0, -1, 1, -1, 0, 1], [1, 0

In [164]:
length(v5)

221

In [179]:
@time (ar, ir)=allprobsn(xs, ss,11, v4)

  1.988904 seconds (22.09 M allocations: 1.967 GiB, 41.91% gc time)


(Dict{Array,Float64}([1, 1, 0, 0, 0, 0, 0, -1, 0, 0, 0]=>0.00420168,[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]=>0.00420168,[1, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0]=>0.00420168,[0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1]=>0.00840336,[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]=>0.00420168,[0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0]=>0.00420168,[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]=>0.00840336,[-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1]=>0.00420168,[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]=>0.00840336,[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]=>0.00840336…), Set(Any[[0, -1, -1, 0, -1, -1, 0, 0, 0, 0, -1], [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -1], [-1, -1, 0, 0, 0, 1, 0, 0, 0, 0, 1], [1, 0, 0, 0, 1, 0, 0, 0, 0, -1, 0], [0, -1, 0, 0, 0, 1, 0, 0, 0, -1, 0], [0, 0, -1, -1, 0, 0, 0, 0, -1, -1, 0], [-1, 0, 0, 0, -1, -1, 0, -1, -1, 0, 0], [-1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 1, 1, 0, 0, 0, -1, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0]  …  [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, -1, -1, 0, 0, 0, 0, -1], [-1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0], [1, 1, 0, 0, 0, 

In [181]:
v3

Set(Any[[1, -1, -1], [-1, 1, 1], [-1, 1, 0], [1, 0, -1], [1, 0, 1], [-1, -1, 1], [1, 1, 1], [1, -1, 0], [0, 1, -1], [-1, 0, 1], [1, -1, 1], [1, 1, -1], [0, -1, 1], [-1, 1, -1], [-1, -1, -1]])

In [150]:
length(v)+length(p)

238

In [124]:
?collect

search: collect CoolLexCombinations get_color_palette



```
collect(element_type, collection)
```

Return an `Array` with the given element type of all items in a collection or iterable. The result has the same shape and number of dimensions as `collection`.

# Examples

```jldoctest
julia> collect(Float64, 1:2:5)
3-element Array{Float64,1}:
 1.0
 3.0
 5.0
```

---

```
collect(collection)
```

Return an `Array` of all items in a collection or iterator. For dictionaries, returns `Pair{KeyType, ValType}`. If the argument is array-like or is an iterator with the [`HasShape`](@ref IteratorSize) trait, the result will have the same shape and number of dimensions as the argument.

# Examples

```jldoctest
julia> collect(1:2:13)
7-element Array{Int64,1}:
  1
  3
  5
  7
  9
 11
 13
```


In [ ]:
function shanonH_n(xxs, ss, n)
    probs=allprobsn(xxs,ss,n)
    h=0
    for q in values(probs)
        if q>0.00001
            h+=q*log(q)
        end
    end
    result=-h
    return result
end

In [ ]:
0==0.000


In [ ]:
nm=div(nmax,2)
hhsn=zeros(h,w,nm)

@time for n=1:7
    for j=1:h, k=1:w
        xs=csdsymb[j,k,:]
      
        hhsn[j,k,n]=shanonH_n(xs, ss, n)
    end
end

In [ ]:
heatmap(hhsn[:,:,3], aspect_ratio=1)

In [ ]:
hhsn[:,:,1]==hhsn[:,:,4]

In [ ]:
allprobsn(xs, ss, 3)

In [ ]:
ss=[-1,0,1]
bla=[[q, j] for q in ss, j in ss ]

In [45]:
xs=csdsymb[16,42,:];
ys=[1]

1-element Array{Int64,1}:
 1

In [46]:
problock(xs, [1]) , problock(xs, [0]),problock(xs, [-1])

(0.016129032258064516, 0.9556451612903226, 0.028225806451612902)

In [ ]:
heatmap(sumsym, aspect_ratio=1)

In [ ]:
?heatmap

In [68]:
a=[0,1]
n=[0, 1]
z=[1,1,0]
z2=[1,0,1]

3-element Array{Int64,1}:
 1
 0
 1

In [71]:
@time issubvec(a, z2)

  0.000008 seconds (9 allocations: 512 bytes)


true

In [72]:
@inbounds

LoadError: MethodError: no method matching @inbounds(::LineNumberNode, ::Module)
Closest candidates are:
  @inbounds(::LineNumberNode, ::Module, !Matched::Any) at essentials.jl:538

In [60]:
as=string(a)

"[0, 1]"